# Stage 0 Data Exploration & Preprocessing (Vanilla)

Stage 0 data exploration will aim to yield an introductory understanding of the Santander Value Prediction dataset.

## Load Libraries:

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import pickle as pkl

import matplotlib.pyplot as plt
import seaborn as sns

import os
import pdb
import h5py

from sklearn.decomposition import PCA

## Load the Data:

In [ ]:
data_path = '../data/'
train_df = pd.read_csv(data_path + 'train.csv', index_col='ID')
test_df = pd.read_csv(data_path + 'test.csv', index_col='ID')

In [ ]:
print('Shape of train dataframe: {} Rows, {} Columns'.format(*train_df.shape))
print('Shape of test dataframe: {} Rows, {} Columns'.format(*test_df.shape))

In [ ]:
# Separate the training labels
labels = train_df.target
train_df.drop(columns=['target'], inplace=True)

In [ ]:
train_df.info()

In [ ]:
test_df.info()

## Dataset Exploration:

In [ ]:
# Find number of null values 
print 'Number of null values in training set:', train_df.isnull().sum().sum()
print 'Number of null values in test set:', test_df.isnull().sum().sum()

In [ ]:
# Find sparsity of dataset
def how_sparse(df):
    num_nonzero = df.astype(bool).sum(axis=0).sum()
    return(100 * num_nonzero/float(df.shape[0] * df.shape[1]))

In [ ]:
print 'Sparsity of training data (in percent):', how_sparse(train_df)
print 'Sparsity of test data (in percent):', how_sparse(test_df)

In [ ]:
# How many columns have constant values? 
def is_constant(df):
    constant = []
    for column in df.columns.values:
        if len(df[column].unique()) == 1:
            constant.append(column)
    return(constant)

In [ ]:
print 'Number of columns with constant values in train dataset:', len(is_constant(train_df))
print 'Number of columns with constant values in test dataset:', len(is_constant(test_df))

Interestingly, there are 256 columns in the training dataset that have constant values while there are 0 columns in the test dataset. This behavior is most likely due to the test dataset having almost 10x more samples than the training dataset. During the data preprocessing stage, the 256 columns with constant values will need to be removed from both the training and test sets.

In [ ]:
drop_cols = is_constant(train_df)

In [ ]:
# Drop constant columns
train_df.drop(columns=drop_cols, axis=1, inplace=True)
test_df.drop(columns=drop_cols, axis=1, inplace=True)

In [ ]:
print('Shape of training dataset: {} Rows, {} Columns'.format(*train_df.shape))
print('Shape of test dataset: {} Rows, {} Columns'.format(*test_df.shape))

In [ ]:
# Finding duplicate columns
def get_duplicates(df):
    groups = df.columns.to_series().groupby(df.dtypes).groups
    dups = []
    
    for t, v in groups.items():
        cs = df[v].columns
        vs = df[v]
        lcs = len(cs)
        
        for i in range(lcs):
            ia = vs.iloc[:, i].values
            for j in range(i+1, lcs):
                ja = vs.iloc[:, j].values
                if np.array_equal(ia, ja):
                    dups.append(cs[i])
    return dups

In [ ]:
duplicate_cols = get_duplicates(train_df)

In [ ]:
print 'Duplicate columns:\n', set(duplicate_cols)

In [ ]:
# Remove duplicate columns
train_df.drop(list(set(duplicate_cols)), axis=1, inplace=True)
test_df.drop(list(set(duplicate_cols)), axis=1, inplace=True)

In [ ]:
print('Shape of training dataset: {} Rows, {} Columns'.format(*train_df.shape))
print('Shape of test dataset: {} Rows, {} Columns'.format(*test_df.shape))

### Correlation Analysis:

In [ ]:
label_df = pd.DataFrame(data=labels)
labeled_train = pd.concat([train_df, label_df], axis=1)
train_corr = labeled_train.corr()

In [ ]:
# Features most highly correlated with the label
display(train_corr['target'].sort_values(ascending=False)[1:11])

The list of features above are the training features that are most-correlated with the training labels. Intuitively, I expect these features to be the most important ones.

In [ ]:
# Save h5 files
def saveh5(fname, data, dname):
    h5f = h5py.File(fname, 'w')
    h5f.create_dataset(dname, data=data)
    h5f.close()
    return None

In [ ]:
# Save pickle file
def savepickle(fname, data):
    with open(fname, 'wb') as handle:
        pkl.dump(data, handle)

In [ ]:
# Save preprocessed dataset (constant columns and duplicate columns removed):
saveh5('../data/train_s0_vanilla.h5', labeled_train, 'train_s0')
saveh5('../data/test_s0_vanilla.h5', test_df, 'test_s0')

In [ ]:
# Save training and test indexes
savepickle('../data/train_idx.pkl', labeled_train.index.values)
savepickle('../data/test_idx.pkl', test_df.index.values)

In [ ]:
# Save training and test column names
savepickle('../data/train_cols.pkl', labeled_train.columns.values)
savepickle('../data/test_cols.pkl', test_df.columns.values)

### PCA:
Performing PCA on the combined training and test set will allow me to visualize the percentage of variance explained for all features in the dataset. By determining which features don't contribute much information, I can further trim the size of the dataset.

In [ ]:
all_data = pd.concat([train_df, test_df], axis=0)
print('Shape of all data: {} Rows, {} Columns'.format(*all_data.shape))

In [ ]:
# Fit PCA model
pca = PCA()
pca.fit(all_data)

In [ ]:
ratios = np.cumsum(pca.explained_variance_ratio_)

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(ratios)
plt.title('Explained Variance Plot for All Data')
plt.xlabel('Number of Components')
plt.yticks(np.arange(0, 1.2, step=0.1))
plt.grid(True)
plt.show()

The plot of explained variance for both train and test data show that the first **3000** features account for 95% of the variance in both datasets. This will be information that I'll use in subsequent preprocessing efforts. 

### Distribution of Training Labels:

In [ ]:
plt.figure(figsize=(10, 6))
sns.distplot(np.log1p(labels), color='green', kde=True, bins=100)
plt.title('Distribution of log(target) Values')
plt.xlabel('log(target)')
plt.show()